In [1]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent
# os.chdir(os.path.join(base_dir, 'serverproject'))
os.chdir(base_dir)
print(os.getcwd())

# Load dotenv
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
%load_ext autoreload
%autoreload 2

from destinyapp.models import StreamRecapData, FastRecapData

from core import services
from core import utils
from core import controller

# Start of Prod Controller

In [3]:
video_id="phITCK1LnDs"
cost=0

In [4]:
import time
times=[]
t_start=time.time()
times.append({"start":t_start})

In [5]:
os.getcwd()

'd:\\DestinyFolder\\DestinyRecaps\\DestinyRecapsApi\\serverproject'

In [6]:
# Get and process transcript
await services.video_and_transcript.download_youtube_transcript(video_id)
raw_transcript=await services.video_and_transcript.read_and_process_youtube_transcript(video_id)
transcript, linked_transcript=services.fast.transcript_processing.process_yt_transcript(raw_transcript, video_id)
t_transcript=time.time()
times.append({"transcript":t_transcript})
if (len(utils.api_requests.enc.encode(transcript)))>(120*1000):
    raise Exception("Transcript too long")

download thread closed
d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [14]:
os.getcwd()

'd:\\DestinyFolder\\DestinyRecaps\\DestinyRecapsApi\\serverproject'

In [16]:
os.path.exists(r"D:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject\destinyapp\working_folder\transcripts\FwtbXklEItc_trancript.en.srt")

True

In [13]:
print(os.path.exists('destinyapp/working_folder/transcripts/FwtbXklEItc_transcript.en.srt'))

False


In [7]:
# Make Topics
topics_str, major_topics, temp_cost=await services.fast.data_gen.create_topics(transcript)
t_topics=time.time()
times.append({"topics":t_topics})
cost+=temp_cost

In [8]:
# Annotate Text batches
text_chunks_no_overlap = await services.stream_plot.data_gen.create_text_chunks(transcript, 0)
text_chunk_batches = await services.stream_plot.data_gen.generate_text_chunk_batches(text_chunks_no_overlap)
responses, annotated_results, temp_cost = await services.fast.data_gen.annotate_all_batches(text_chunk_batches, topics_str)
cost+=temp_cost
t_annotations=time.time()
times.append({"annotations":t_annotations})

Cost:  0.000447
Results:  6


In [9]:
# Create Plot
annotated_segments, category_locations = await services.stream_plot.data_processing.create_segments(linked_transcript, annotated_results, major_topics, transcript)
plot_segments=await services.stream_plot.data_processing.annotated_to_plot_segments(annotated_segments)
plot_object=await services.stream_plot.data_processing.create_plot_object(plot_segments, category_locations, video_id)
t_plot_object=time.time()
times.append({"plot_object":t_plot_object})

5
Material and part improvements
Control and steering enhancements
Testing and performance evaluation
Damage assessment and recovery


In [10]:
# Annotate Extra
plot_object, temp_cost=await services.stream_plot.extra_annotations.recap_segments(plot_object)
cost+=temp_cost
t_recap_segments=time.time()
times.append({"recap_segments":t_recap_segments})
plot_object, temp_cost=await services.stream_plot.extra_annotations.recap_abstractions(plot_object)
cost+=temp_cost
t_recap_abstractions=time.time()
times.append({"recap_abstractions":t_recap_abstractions})

In [11]:
print("Created Fast Recap for video_id: ", video_id, " with cost: ", cost)

for i in range(1,len(times)):
    print(list(times[i].keys())[0], ": ", list(times[i].values())[0]-list(times[i-1].values())[0])

print("Total Time: ", list(times[-1].values())[0]-list(times[0].values())[0])

Created Fast Recap for video_id:  phITCK1LnDs  with cost:  0.0015850500000000002
transcript :  15.421881437301636
topics :  2.763185977935791
annotations :  2.3190062046051025
plot_object :  0.12117791175842285
recap_segments :  0.9868555068969727
recap_abstractions :  0.9040377140045166
Total Time:  22.51614475250244
